In [1]:
import uuid
import base64
import csv
import hashlib
import numpy as np
import pandas as pd
from ast import literal_eval

In [2]:
def to_uuid(s_id):
    return str(uuid.UUID(bytes=base64.urlsafe_b64decode(s_id + '==')))

def string_to_uuid(s):
    s = s.encode('utf-8')
    out = hashlib.md5(s).hexdigest()
    return out[:8] + '-' + out[8:12] + '-' + out[12:16] + '-' + out[16:20] + '-' + out[20:]

In [63]:
with open("../files/albums_filtered.csv", 'r') as f_read, \
    open("../files/sql/albums_insert.csv", 'w') as f_write, \
    open("../files/sql/artists_albums.csv", 'w') as f_rel:
    reader = csv.DictReader(f_read)
    
    names = ['id', 'name', 'type', 'release_date', 'total_tracks']
    writer = csv.DictWriter(f_write, names)
    writer.writeheader()
    
    rel_names = ['artist', 'album']
    rel_writer = csv.DictWriter(f_rel, rel_names)
    rel_writer.writeheader()

    count = 0
    
    for line in reader:
        line['id'] = to_uuid(line['id'])
        
        artists = literal_eval(line['artists'])
        for artist in artists:
            rel_writer.writerow({'artist': to_uuid(artist), 'album': line['id']})
        del line['markets']
        del line['artists']
        
        try:
            line['release_date'] = str(pd.to_datetime(line['release_date']).date())
        except Exception:
            line['release_date'] = None
        
        writer.writerow(line)
        count += 1
        print(count, end='\r')

In [52]:
with open("../files/songs_merged.csv", 'r') as f_read, \
    open("../files/sql/songs_insert.csv", 'w') as f_write, \
    open("../files/sql/artists_songs.csv", 'w') as f_rel:
    reader = csv.DictReader(f_read)
    
    names = ['id', 'name', 'album', 'duration_ms', 'explicit', 'disc_number', 'track_number', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    writer = csv.DictWriter(f_write, names)
    writer.writeheader()
    
    rel_names = ['artist', 'song']
    rel_writer = csv.DictWriter(f_rel, rel_names)
    rel_writer.writeheader()
    
    count = 0
    
    for line in reader:
        line['id'] = to_uuid(line['id'])
        line['album'] = to_uuid(line['album'])
        
        artists = literal_eval(line['artists'])
        for artist in artists:
            rel_writer.writerow({'artist': to_uuid(artist), 'song': line['id']})
        del line['artists']
        
        writer.writerow(line)
        count += 1
        print(count, end='\r')

In [3]:
edges = set()
genre_dict = {}

with open("../files/artists_filtered.csv", 'r') as f_read, \
    open("../files/sql/artists_insert.csv", 'w') as f_write, \
    open("../files/sql/artists_genres.csv", 'w') as f_rel:
    reader = csv.DictReader(f_read)
    
    names = ['id', 'name', 'followers']
    writer = csv.DictWriter(f_write, names)
    writer.writeheader()
    
    rel_names = ['artist', 'genre']
    rel_writer = csv.DictWriter(f_rel, rel_names)
    rel_writer.writeheader()
        
    count = 0
    for line in reader:
        line['id'] = to_uuid(line['id'])
        line['followers'] = int(float(line['followers']))
        
        artists = literal_eval(line['related_artists'])
        for artist in artists:
            artist = to_uuid(artist)
            s = frozenset((line['id'], artist))
            edges.add(s)
        del line['related_artists']
        
        genres = literal_eval(line['genres'])
        for genre in genres:
            if genre not in genre_dict:
                genre_dict[genre] = string_to_uuid(genre)
            genre_id = genre_dict[genre]
            rel_writer.writerow({'artist': line['id'], 'genre': genre_id})
        del line['genres']
        
        writer.writerow(line)
        count += 1
        print(count, end='\r')
        
with open("../files/sql/artists_artists.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['source', 'target'])
    for edge in edges:
        writer.writerow(edge)
        
del edges

with open("../files/sql/genres_insert.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'name'])
    for row in genre_dict.items():
        writer.writerow([row[1], row[0]])
        
del genre_dict